# Adaptive sampling

In this tutorial, we will showcase how to use adaptive sampling simulations on a molecular system. The sample system in this case is the NTL9 protein.

Let's import HTMD and do some definitions:

In [1]:
from htmd.ui import *


Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/
To update: conda update htmd -c acellera -c psi4

You are on the latest HTMD version (unpackaged : /home/joao/maindisk/software/repos/Acellera/htmd/htmd).



## Get the generators folder structure

Get the data for this tutorial [here](http://pub.htmd.org/tutorials/adaptive-sampling/generators.tar.gz). Alternatively, you can download the data using `wget`:

In [2]:
import os, glob
assert os.system('wget -rcN -np -nH -q --cut-dirs=2 -R index.html* http://pub.htmd.org/tutorials/adaptive-sampling/generators/') == 0
for file in glob.glob('./generators/*/run.sh'):
    os.chmod(file, 0o755)

In [3]:
!tree generators | head -20

generators
|-- ntl9_1ns_0
|   |-- input
|   |-- input.coor
|   |-- input.xsc
|   |-- parameters
|   |-- run.sh
|   |-- structure.pdb
|   `-- structure.psf
|-- ntl9_1ns_1
|   |-- input
|   |-- input.coor
|   |-- input.xsc
|   |-- parameters
|   |-- run.sh
|   |-- structure.pdb
|   `-- structure.psf
`-- ntl9_1ns_2
    |-- input
    |-- input.coor


## Adaptive classes

HTMD has two types of adaptive sampling:

* AdaptiveMD (free exploration)
* AdaptiveGoal (exploration + exploitation)

Create a directory for each type of adaptive and copy the generators into them:

In [4]:
os.makedirs('./adaptivemd', exist_ok=True)
os.makedirs('./adaptivegoal', exist_ok=True)
shutil.copytree('./generators', './adaptivemd/generators')
shutil.copytree('./generators', './adaptivegoal/generators')

'./adaptivegoal/generators'

## AdaptiveMD

Let's change directory to the `adaptivemd` one and work there:

In [5]:
os.chdir('./adaptivemd')

* Setup the queue that will be used for simulations. 
* Tell it to store completed trajectories in the data folder as this is where `AdaptiveMD` expects them to be by default

In [6]:
queue = LocalGPUQueue()
queue.datadir = './data'

In [7]:
ad = AdaptiveMD()
ad.app = queue

* Set the `nmin`, `nmax` and `nepochs`

In [8]:
ad.nmin = 1
ad.nmax = 3
ad.nepochs = 3

* Choose what projection to use for the construction of the Markov model

In [9]:
protsel = 'protein and name CA'
ad.projection = MetricSelfDistance(protsel)

* Set the `updateperiod` of the Adaptive to define how often it will poll for completed simulations and redo the analysis

In [10]:
ad.updateperiod = 120 # execute every 2 minutes

Launch the `AdaptiveMD` run:

In [11]:
ad.run()

2018-03-19 11:46:45,536 - htmd.adaptive.adaptive - INFO - Processing epoch 0
2018-03-19 11:46:45,538 - htmd.adaptive.adaptive - INFO - Epoch 0, generating first batch
2018-03-19 11:46:45,563 - htmd.queues.localqueue - INFO - Trying to determine all GPU devices
2018-03-19 11:46:45,614 - htmd.queues.localqueue - INFO - Using GPU devices 0,1,2,3
2018-03-19 11:46:45,941 - htmd.queues.localqueue - INFO - Trying to determine all GPU devices
2018-03-19 11:46:45,993 - htmd.queues.localqueue - INFO - Using GPU devices 0,1,2,3
2018-03-19 11:46:45,997 - htmd.queues.localqueue - INFO - Queueing /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivemd/input/e1s1_ntl9_1ns_0
2018-03-19 11:46:45,999 - htmd.queues.localqueue - INFO - Queueing /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivemd/input/e1s2_ntl9_1ns_1
2018-03-19 11:46:45,999 - htmd.queues.localqueue - INFO - Running /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivemd/input/e1s1_ntl9_1ns_0 o

A Jupyter Widget

/home/joao/maindisk/SANDBOX/miniconda3/miniconda3/lib/python3.6/site-packages/pyemma/__init__.py:91: UserWarning: You are not using the latest release of PyEMMA. Latest is 2.5.1, you have 2.4.
  .format(latest=latest, current=current), category=UserWarning)


A Jupyter Widget

2018-03-19 11:50:50,288 - htmd.metricdata - INFO - Dropped 0 trajectories from 3 resulting in 3


19-03-18 11:50:50 pyemma.msm.estimators.implied_timescales.ImpliedTimescales[2] WARNING  Changed user setting nits to the number of available timescales nits=7


2018-03-19 11:50:50,301 - pyemma.msm.estimators.implied_timescales.ImpliedTimescales[2] - WARNING - Changed user setting nits to the number of available timescales nits=7
2018-03-19 11:50:50,636 - htmd.model - WARNING - PCCA returned empty macrostates. Reducing the number of macrostates to 3.
2018-03-19 11:50:50,678 - htmd.model - WARNING - PCCA returned empty macrostates. Reducing the number of macrostates to 2.
2018-03-19 11:50:50,688 - htmd.model - INFO - 93.3% of the data was used
2018-03-19 11:50:50,690 - htmd.model - INFO - Number of trajectories that visited each macrostate:
2018-03-19 11:50:50,691 - htmd.model - INFO - [3 3]
2018-03-19 11:50:50,692 - htmd.model - INFO - Take care! Macro 0 has been visited only in 3 trajectories:
2018-03-19 11:50:50,693 - htmd.model - INFO - 
simid = 0
parent = 0
input = None
trajectory = ['filtered/e1s1_ntl9_1ns_0/output.filtered.xtc']
molfile = filtered/filtered.pdb
numframes = [None]

2018-03-19 11:50:50,694 - htmd.model - INFO - 
simid = 1
p

A Jupyter Widget

A Jupyter Widget

2018-03-19 11:54:54,633 - htmd.metricdata - INFO - Dropped 0 trajectories from 6 resulting in 6
2018-03-19 11:54:56,750 - htmd.model - INFO - 100.0% of the data was used
2018-03-19 11:54:56,753 - htmd.model - INFO - Number of trajectories that visited each macrostate:
2018-03-19 11:54:56,753 - htmd.model - INFO - [1 1 1 2 3 4 6 6]
2018-03-19 11:54:56,755 - htmd.model - INFO - Take care! Macro 0 has been visited only in 1 trajectories:
2018-03-19 11:54:56,755 - htmd.model - INFO - 
simid = 0
parent = 0
input = None
trajectory = ['filtered/e1s1_ntl9_1ns_0/output.filtered.xtc']
molfile = filtered/filtered.pdb
numframes = [10]

2018-03-19 11:54:56,756 - htmd.model - INFO - Take care! Macro 1 has been visited only in 1 trajectories:
2018-03-19 11:54:56,757 - htmd.model - INFO - 
simid = 0
parent = 0
input = None
trajectory = ['filtered/e1s1_ntl9_1ns_0/output.filtered.xtc']
molfile = filtered/filtered.pdb
numframes = [10]

2018-03-19 11:54:56,763 - htmd.model - INFO - Take care! Macro 2 has 

## AdaptiveGoal

Now let's change to the `adaptivegoal` directory and work there instead:

In [12]:
os.chdir('../adaptivegoal')

* Most of the class arguments are identical to AdaptiveMD

In [13]:
adg = AdaptiveGoal()
adg.app = queue
adg.nmin = 1
adg.nmax = 3
adg.nepochs = 2
adg.generatorspath = './generators'
adg.projection = MetricSelfDistance('protein and name CA')
adg.updateperiod = 120  # execute every 2 minutes
adg.goalfunction = None  # set to None just as an example

* It requires the `goalfunction` argument which defines a goal
* We can define a variety of different goal functions

## The goal function

The goal function will:
* take as input a `Molecule` object of a simulation and 
* produce as output a score for each frame of that simulation. 
* The higher the score, the more desirable that simulation frame for being respawned.

### RMSD goal function

For this goal function, we will use a crystal structure of NTL9.

You can download the structure from the following link and save it on the `adaptivegoal` directory:

* [NTL9 crystal structure](http://pub.htmd.org/tutorials/adaptive-sampling/ntl9_crystal.pdb).

Alternatively, you can download the structure using `wget`.

In [14]:
assert os.system('wget -q http://pub.htmd.org/tutorials/adaptive-sampling/ntl9_crystal.pdb') == 0

We can define a simple goal function that uses the RMSD between the conformation sampled and a reference (in this case, the crystal structure), and returns a score to be evaluated by the `AdaptiveGoal` algorithm:

In [15]:
ref = Molecule('./ntl9_crystal.pdb')

def mygoalfunction(mol):
    rmsd = MetricRmsd(ref, 'protein and name CA').project(mol)
    return -rmsd  # or even 1/rmsd

adg.goalfunction = mygoalfunction

`AdaptiveGoal` ranks conformations from a high to low score. For the case of RMSD, since we want lower RMSD to give higher score, the symetric value is returned instead (the inverse would also work).

Launch the `AdaptiveGoal` run:

In [16]:
adg.run()

2018-03-19 12:00:20,987 - htmd.adaptive.adaptive - INFO - Processing epoch 0
2018-03-19 12:00:20,989 - htmd.adaptive.adaptive - INFO - Epoch 0, generating first batch
2018-03-19 12:00:21,013 - htmd.queues.localqueue - INFO - Queueing /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s1_ntl9_1ns_0
2018-03-19 12:00:21,014 - htmd.queues.localqueue - INFO - Running /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s1_ntl9_1ns_0 on device 2
2018-03-19 12:00:21,015 - htmd.queues.localqueue - INFO - Queueing /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s2_ntl9_1ns_1
2018-03-19 12:00:21,018 - htmd.queues.localqueue - INFO - Running /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s2_ntl9_1ns_1 on device 0
2018-03-19 12:00:21,018 - htmd.queues.localqueue - INFO - Queueing /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s3_ntl9_1ns_2
2018-0

A Jupyter Widget

A Jupyter Widget

2018-03-19 12:04:25,262 - htmd.metricdata - INFO - Dropped 0 trajectories from 3 resulting in 3
Projecting trajectories: 100%|██████████| 3/3 [00:00<00:00, 11.17it/s]
2018-03-19 12:04:26,086 - htmd.projections.metric - INFO - Frame step 0.10000000149011612ns was read from the trajectories. If it looks wrong, redefine it by manually setting the MetricData.fstep property.
2018-03-19 12:04:26,294 - htmd.model - INFO - 100.0% of the data was used
2018-03-19 12:04:26,296 - htmd.model - INFO - Number of trajectories that visited each macrostate:
2018-03-19 12:04:26,297 - htmd.model - INFO - [2 3 2]
2018-03-19 12:04:26,299 - htmd.model - INFO - Take care! Macro 0 has been visited only in 2 trajectories:
2018-03-19 12:04:26,300 - htmd.model - INFO - 
simid = 0
parent = 0
input = None
trajectory = ['filtered/e1s1_ntl9_1ns_0/output.filtered.xtc']
molfile = filtered/filtered.pdb
numframes = [None]

2018-03-19 12:04:26,300 - htmd.model - INFO - 
simid = 2
parent = 2
input = None
trajectory = ['fil

### Functions with multiple arguments

The goal function can also take multiple arguments. This allows flexibility and on-the-fly comparisons to non-static conformations (i.e. compare with different references as the run progresses). Here, we redefine the previous goal function with multiple arguments:

In [17]:
def newgoalfunction(mol, crystal):
    rmsd = MetricRmsd(crystal, 'protein and name CA').project(mol)
    return -rmsd  # or even 1/rmsd

Now we clean the previous `AdaptiveGoal` run, and start a new one with the new goal function:

In [18]:
# clean previous run
shutil.rmtree('./input')
shutil.rmtree('./data')
shutil.rmtree('./filtered')

# run with new goal
ref = Molecule('./ntl9_crystal.pdb')
adg.goalfunction = (newgoalfunction, (ref,))
adg.run()

2018-03-19 12:08:41,639 - htmd.adaptive.adaptive - INFO - Processing epoch 0
2018-03-19 12:08:41,640 - htmd.adaptive.adaptive - INFO - Epoch 0, generating first batch
2018-03-19 12:08:41,659 - htmd.queues.localqueue - INFO - Queueing /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s1_ntl9_1ns_0
2018-03-19 12:08:41,660 - htmd.queues.localqueue - INFO - Queueing /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s2_ntl9_1ns_1
2018-03-19 12:08:41,660 - htmd.queues.localqueue - INFO - Running /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s1_ntl9_1ns_0 on device 3
2018-03-19 12:08:41,662 - htmd.queues.localqueue - INFO - Running /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s2_ntl9_1ns_1 on device 0
2018-03-19 12:08:41,662 - htmd.queues.localqueue - INFO - Queueing /data/joao/maindisk/software/repos/Acellera/htmd/tutorials/adaptivegoal/input/e1s3_ntl9_1ns_2
2018-0

A Jupyter Widget

A Jupyter Widget

2018-03-19 12:12:45,684 - htmd.metricdata - INFO - Dropped 0 trajectories from 3 resulting in 3
Projecting trajectories: 100%|██████████| 3/3 [00:00<00:00, 12.06it/s]
2018-03-19 12:12:46,436 - htmd.projections.metric - INFO - Frame step 0.10000000149011612ns was read from the trajectories. If it looks wrong, redefine it by manually setting the MetricData.fstep property.
2018-03-19 12:12:46,669 - htmd.model - INFO - 96.7% of the data was used
2018-03-19 12:12:46,670 - htmd.model - INFO - Number of trajectories that visited each macrostate:
2018-03-19 12:12:46,671 - htmd.model - INFO - [1 1 3 3]
2018-03-19 12:12:46,673 - htmd.model - INFO - Take care! Macro 0 has been visited only in 1 trajectories:
2018-03-19 12:12:46,673 - htmd.model - INFO - 
simid = 1
parent = 1
input = None
trajectory = ['filtered/e1s2_ntl9_1ns_1/output.filtered.xtc']
molfile = filtered/filtered.pdb
numframes = [None]

2018-03-19 12:12:46,674 - htmd.model - INFO - Take care! Macro 1 has been visited only in 1 trajec

### Other goal function examples

HTMD includes other two goal functions: The secondary structure goal function and the contacts goal function.

#### Secondary structure goal function

In [19]:
ref = Molecule('./ntl9_crystal.pdb')

def ssGoal(mol, crystal):
    crystalSS = MetricSecondaryStructure().project(crystal)[0]
    proj = MetricSecondaryStructure().project(mol)
    # How many crystal SS match with simulation SS
    ss_score = np.sum(proj == crystalSS, axis=1) / proj.shape[1]  
    return ss_score

adg.goalfunction = (ssGoal, (ref,))

#### Contacts goal function

In [20]:
ref = Molecule('./ntl9_crystal.pdb')

def contactGoal(mol, crystal):
    crystalCO = MetricSelfDistance('protein and name CA', pbc=False,
                                   metric='contacts', 
                                   threshold=10).project(crystal)
    proj = MetricSelfDistance('protein and name CA', 
                              metric='contacts', 
                              threshold=10).project(mol)
    # How many crystal contacts are seen?
    co_score = np.sum(proj[:, crystalCO] == 1, axis=1)
    co_score /= np.sum(crystalCO)
    return ss_score

adg.goalfunction = (contactGoal, (ref,))

Many more goal functions can be devised.